In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, classification_report, multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

## data adapted

In [202]:


# Cargar CSV (usa sep=';' porque tu dataset está separado por punto y coma)
df = pd.read_csv("../data/dataset.csv", sep=";")

# Crear columna de etiquetas como listas
df['labels'] = df['group'].str.split('|')

# Lista de todas las clases únicas
all_labels = sorted(set(label for labels in df['labels'] for label in labels))
print(all_labels)
df.drop("group",inplace=True,axis=1)


['cardiovascular', 'hepatorenal', 'neurological', 'oncological']


## Gemini model

In [205]:


# Cargar variables desde .env
load_dotenv()

# Configurar Gemini con la API Key
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))


In [206]:

# Función para generar embeddings con Gemini
def get_gemini_embedding(text):
    model = "models/text-embedding-004"
    if not text or not isinstance(text, str):
        return [0.0] * 768 # Tamaño del embedding para este modelo
    try:
        response = genai.embed_content(model=model, content=text, task_type="classification")
        return response["embedding"]
    except Exception as e:
        print(f"Error generando embedding para el texto: {text[:50]}... Error: {e}")
        return [0.0] * 768


In [ ]:

# --- PASO 1: GENERAR EMBEDDINGS ---
print("Generando embeddings para las etiquetas...")
# Convertir a minúsculas para consistencia
label_embeddings = {label: get_gemini_embedding(label) for label in all_labels}

# Crear texto combinado y manejar nulos
df["text"] = df["title"].fillna('') + " " + df["abstract"].fillna('')

print("Generando embeddings para los artículos... (esto puede tardar)")
df["embedding"] = df["text"].apply(get_gemini_embedding)

# --- PASO 2: CALCULAR SIMILITUD Y APLICAR LÓGICA MULTIETIQUETA ---
X = np.array(df["embedding"].to_list())
label_matrix = np.array(list(label_embeddings.values()))

# Calcular similitudes
similarities = cosine_similarity(X, label_matrix)

mlb = MultiLabelBinarizer(classes=all_labels)
y_test= mlb.fit_transform(df['labels'])


Generando embeddings para las etiquetas...
Generando embeddings para los artículos... (esto puede tardar)


## Umbrales

In [ ]:
from sklearn.metrics import f1_score

best_thresholds = {}
y_pred = np.zeros_like(similarities)

# Iterar sobre cada clase (columna)
for i, label in enumerate(all_labels):
    best_f1_class = -1
    best_threshold_class = 0
    
    # Extraer las etiquetas verdaderas y las similitudes para esta clase
    y_true_class = y_true_binarized[:, i]
    similarities_class = similarities[:, i]
    
    # Buscar el mejor umbral solo para esta clase
    for threshold in np.arange(0.4, 0.7, 0.01):
        y_pred_class = (similarities_class >= threshold).astype(int)
        f1 = f1_score(y_true_class, y_pred_class, zero_division=0)
        if f1 > best_f1_class:
            best_f1_class = f1
            best_threshold_class = threshold
            
    best_thresholds[label] = best_threshold_class
    print(f"Mejor umbral para {label}: {best_threshold_class:.2f} (F1: {best_f1_class:.4f})")
    
    # Aplicar el mejor umbral encontrado para esta clase en la matriz de predicción final
    y_pred[:, i] = (similarities_class >= best_thresholds[label]).astype(int)

# Ahora evalúa y genera el CSV usando 'y_pred_binarized_final'

Mejor umbral para cardiovascular: 0.54 (F1: 0.6466)
Mejor umbral para hepatorenal: 0.53 (F1: 0.7083)
Mejor umbral para neurological: 0.40 (F1: 0.6673)
Mejor umbral para oncological: 0.58 (F1: 0.4801)


## Modelo

In [ ]:


# Tus embeddings son las características (X) y las etiquetas binarizadas son el objetivo (y)
X = np.array(df["embedding"].to_list())
y = y_test

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Usar un clasificador que soporte multietiqueta, como OneVsRest con LogisticRegression
# OneVsRest entrena un clasificador binario para cada clase, lo cual es perfecto para tu problema
clf = OneVsRestClassifier(LogisticRegression(solver='liblinear',class_weight='balanced'))
clf.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

KeyError: 'embedding'

## Evaluacion de resultado

In [ ]:
# Calcular F1-Score Ponderado (métrica principal del challenge)
weighted_f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Métrica Principal - F1 Score Ponderado (Weighted): {weighted_f1:.4f}\n")

# Reporte de clasificación completo (muestra métricas por cada clase)
print("Reporte de Clasificación Detallado (por clase):")
# Usamos `target_names` para que el reporte muestre los nombres de las clases
print(classification_report(y_true_binarized, y_pred_binarized, target_names=all_labels))


Métrica Principal - F1 Score Ponderado (Weighted): 0.6475

Reporte de Clasificación Detallado (por clase):
                precision    recall  f1-score   support

cardiovascular       0.50      0.93      0.65      1268
   hepatorenal       0.68      0.74      0.71      1091
  neurological       0.50      1.00      0.67      1785
   oncological       0.34      0.80      0.48       601

     micro avg       0.50      0.90      0.64      4745
     macro avg       0.50      0.87      0.63      4745
  weighted avg       0.52      0.90      0.65      4745
   samples avg       0.57      0.92      0.66      4745



In [ ]:
mcm = multilabel_confusion_matrix(y_test,y_pred)
for i, label in enumerate(all_labels):
    cm = mcm[i]
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No", "Yes"], yticklabels=["No", "Yes"])
    plt.title(f"Confusion Matrix for {label}")
    plt.ylabel("True")
    plt.xlabel("Predicted")
    plt.show()

## guardar resultado

In [ ]:
print("\n--- GENERANDO ARCHIVO DE SALIDA ---")

# 1. Convertir predicciones binarias a tuplas de texto
predicted_labels_tuples = mlb.inverse_transform(y_pred_binarized)

# 2. Unir las tuplas en un solo string separado por '|'
predicted_labels_strings = ['|'.join(labels) for labels in predicted_labels_tuples]

# 3. Crear el DataFrame de salida final
# Usamos el 'test_df' que guardamos al principio para tener acceso a 'title' y 'abstract'
output_df = df[['title', 'abstract',"labels"]].copy()
output_df['group_predicted'] = predicted_labels_strings

""" # 4. Guardar en un archivo CSV con el formato requerido
output_df.to_csv('predictions_ml_model.csv', index=False, sep=';')
 """

In [181]:
output_df

,title,abstract,group_predicted,labels
0,Adrenoleukodystrophy: survey of 303 cases: bio...,Adrenoleukodystrophy ( ALD ) is a genetically ...,oncological,"[neurological, hepatorenal]"
1,endoscopy reveals ventricular tachycardia secrets,Research question: How does metformin affect c...,cardiovascular|oncological,[neurological]
2,dementia and cholecystitis: organ interplay,Purpose: This randomized controlled study exam...,cardiovascular|hepatorenal|neurological|oncolo...,[hepatorenal]
3,The interpeduncular nucleus regulates nicotine...,Partial lesions were made with kainic acid in ...,cardiovascular|neurological|oncological,[neurological]
4,guillain-barre syndrome pathways in leukemia,Hypothesis: statins improves stroke outcomes v...,cardiovascular|neurological|oncological,[neurological]
...,...,...,...,...
3560,The effect of recombinant human insulin-like g...,We recently demonstrated that recombinant hGH ...,hepatorenal|oncological,"[neurological, cardiovascular, hepatorenal]"
3561,EEG reveals leukemia secrets,Hypothesis: metformin improves dementia outcom...,cardiovascular|neurological|oncological,[neurological]
3562,venous pathways in angina pectoris,Hypothesis: metformin improves heart disease o...,cardiovascular|oncological,"[neurological, cardiovascular]"
3563,Thyroxine abuse: an unusual case of thyrotoxic...,Eating disorders and the associated behavioura...,oncological,[neurological]
